In [1]:
%load_ext autoreload
%autoreload 2

Dependencies

In [25]:
import pathlib as pb

# Environment
ROOT_PATH = pb.Path('..')
DATA_DIR_PATH = ROOT_PATH / 'data'
CACHE_DIR_PATH = ROOT_PATH / '.cache'
TRANSFORMERS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'transformers'
DATASETS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'datasets'
TEST_DATA_FILE = DATA_DIR_PATH / 'test_data.csv'
TRAIN_DATA_FILE = DATA_DIR_PATH / 'train_data.csv'
SUBMISSIONS_DIR_PATH = ROOT_PATH / 'submissions'

# Model Repositories
BERT_RO_MODEL_REPO = 'dumitrescustefan/bert-base-romanian-cased-v1'

In [3]:
import os

os.environ['TRANSFORMERS_CACHE'] = str(TRANSFORMERS_CACHE_DIR_PATH)
os.environ['HF_DATASETS_CACHE'] = str(DATASETS_CACHE_DIR_PATH)
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import torchdata
import torchtext
from torch import Tensor
from torch import nn
from torch import optim
from torch.utils.data import Subset, DataLoader, Dataset
from torch import backends
import typing
import pathlib as pb
import os
import gc
from typing import List, Tuple, Dict, Set
import random
import numpy as np
import pandas as pd
import platform
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score, accuracy_score

/home/invokariman/Projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-processing-2-mFR9zZTH-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from utils import get_available_device, read_data
from preprocess import BertPreprocessor
from data import SexismDataset
from models import BertFlatClassModel

Environment Setup

In [6]:
# Use available GPU
DEVICE: torch.device = get_available_device()

# Deterministic experiments
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
np.random.RandomState(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

Dataset & Preprocessing

In [7]:
train_data_raw, test_data_raw = read_data(DATA_DIR_PATH)

In [8]:
train_dataset = SexismDataset(train_data_raw, BertPreprocessor(train_data_raw, BERT_RO_MODEL_REPO))
test_dataset = SexismDataset(test_data_raw, BertPreprocessor(test_data_raw, BERT_RO_MODEL_REPO))

Model Training

In [9]:
# Training specifications
batch_size = 8
num_epochs = 10
num_workers = 1
learning_rate = 5e-5

In [10]:
# Instantiate language model
# Inspired from: https://luv-bansal.medium.com/fine-tuning-bert-for-text-classification-in-pytorch-503d97342db2
model: nn.Module = torch.compile(BertFlatClassModel(BERT_RO_MODEL_REPO, unfreeze='all')).to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # TODO: weighted loss

Some weights of the model checkpoint at dumitrescustefan/bert-base-romanian-cased-v1 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# Shutup Scikit-Learn Warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [19]:
# Inspired from: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

for i, (train_idx, valid_idx) in enumerate(kf.split(train_dataset)):
    # Split the data
    train_subset = Subset(train_dataset, train_idx[:5000])
    valid_subset = Subset(train_dataset, valid_idx[:1000])

    # Create the dataloaders
    train_loader = DataLoader(
        train_subset,
        batch_size,
    )
    valid_loader = DataLoader(
        train_subset,
        batch_size,
    )

    # ---  Training  ---
    model.train()
    for epoch in range(num_epochs):
        epoch_loss: List[float] = []
        epoch_accy: List[float] = []

        for batch_i, batch in enumerate(train_loader):
            # Send batch to GPU
            batch: Dict[str, Tensor] = { k: v.to(DEVICE) for k, v in batch.items() }

            # Make predictions
            y_true: Tensor | np.ndarray = batch['label']
            y_pred: Tensor | np.ndarray = model.forward(batch)

            # Compute the loss
            optimizer.zero_grad()
            loss: Tensor = loss_fn(y_pred, y_true)
            loss.backward()
            optimizer.step()

            # Compute the accuracy
            y_true = y_true.detach().cpu().numpy()
            y_pred = y_pred.detach().argmax(dim=1).cpu().numpy()
            epoch_accy.append(balanced_accuracy_score(y_true, y_pred))

            # Track progress
            epoch_loss.append(loss.detach().cpu().numpy())

        mean_loss: float = np.array(epoch_loss).mean()
        mean_accy: float = np.array(epoch_accy).mean()
        print('Train Epoch {} - Loss: {}, Accuracy: {}'.format(epoch, mean_loss, mean_accy))

    # --- Validation ---
    model.eval()
    with torch.no_grad():
        epoch_loss = []
        epoch_accy = []

        for batch_i, batch in enumerate(valid_loader):
            # Send batch to GPU
            batch: Dict[str, Tensor] = { k: v.to(DEVICE) for k, v in batch.items() }

            # Make predictions
            y_true: Tensor | np.ndarray = batch['label']
            y_pred: Tensor | np.ndarray = model.forward(batch)

            # Compute the loss
            loss: Tensor = loss_fn(y_pred, y_true)

            # Compute the accuracy
            y_true = y_true.detach().cpu().numpy()
            y_pred = y_pred.detach().argmax(dim=1).cpu().numpy()
            epoch_accy.append(balanced_accuracy_score(y_true, y_pred))

            # Track progress
            epoch_loss.append(loss.detach().cpu().numpy())

        mean_loss = np.array(epoch_loss).mean()
        mean_accy = np.array(epoch_accy).mean()
        print('Validation - Loss: {}, Accuracy: {}'.format(epoch, mean_loss, mean_accy))

[2023-03-25 19:13:00,698] torch._inductor.utils: [WARNING] using triton random, expect difference from eager


Train Epoch 0 - Loss: 0.6729198694229126, Accuracy: 0.5892546031746031


KeyboardInterrupt: 

Prediction

In [37]:
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
labels: List[str] = []

model.eval()
with torch.no_grad():
    for batch_i, batch in enumerate(test_loader):
        # Send batch to GPU
        batch: Dict[str, Tensor] = { k: v.to(DEVICE) for k, v in batch.items() }

        # Make predictions
        y_pred: Tensor | np.ndarray = model.forward(batch)

        # Compute the accuracy
        y_pred = y_pred.detach().argmax(dim=1).cpu().numpy()
        y_pred = np.vectorize(train_dataset.class_to_label.get)(y_pred)

        # Concatenate the results
        labels.extend(y_pred.tolist())

In [36]:
output: pd.DataFrame = pd.DataFrame({ 'Label': pd.Series(data=labels) })
output = output.reset_index()
output = output.rename(columns={ 'index': 'Id' })
output.to_csv(SUBMISSIONS_DIR_PATH / 'submission_5.csv', index=False)

In [ ]:
# Free memory
del model
gc.collect()
torch.cuda.empty_cache()